In [1]:
from pyemd import emd 
from sklearn import mixture
import pickle
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import gensim, math
import time
import pandas as pd
import nltk 
from os.path import dirname, abspath
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import lda
import numpy as np
import gensim
from nltk import word_tokenize
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import string
import os
from scipy.spatial.distance import cosine
from gensim.matutils import softcossim
from scipy.sparse import csc_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import mixture
from collections import Counter
import sklearn.metrics as mt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../Benchmark/GoogleNews-vectors-negative300.bin', binary=True)

# BBC news

In [4]:
class MyDataContainer(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

topics = ["business", "entertainment", "politics", "sport", "tech"]
cwd = dirname(os.getcwd())+"/Data/bbc2/"
BBCNews = MyDataContainer({'data': [],
           'label': []})
 
for i, s in enumerate(topics):
    path = cwd+s
    for filename in os.listdir(path):
        with open(path+"/"+filename, 'r', encoding="latin-1") as f:
            text = f.read() 
            #text = text+(" "+noises[noise_indices[counter]]+" ")*nn if add_noise else text
            BBCNews.data.append(text)
            BBCNews.label.append(i)
        f.close()


In [5]:
seeds_indices = random.sample(range(2225), 222)
noises = [BBCNews.data[t] for t in seeds_indices]
noises = " ".join(noises)
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
noises = noises.translate(translator).lower().split()
noises = list(set(noises))
noises = [t for t in noises if t in word2vec]

# 20 news

In [6]:
cats = [#'comp.graphics',
        #'comp.os.ms-windows.misc',
        'comp.sys.ibm.pc.hardware',
        'rec.autos',
        'rec.motorcycles',
        'rec.sport.baseball',
        'sci.crypt',
        #'sci.electronics',
        'sci.med']
news20_all = fetch_20newsgroups(subset='all',categories=cats)
news20_noise = random.sample(news20_all.data, len(BBCNews.data))
BBCNews_noised = dotdict({"data": [a+b for a, b in zip(news20_noise, BBCNews.data)],
                            "label": BBCNews.label})


In [7]:
BBCNews_noised.data = [t + " " + " ".join(random.sample(noises, 200)) for t in BBCNews.data]

In [8]:
BBCNews_noised.data[0]

'UK economy facing \'major risks\'\n\nThe UK manufacturing sector will continue to face "serious challenges" over the next two years, the British Chamber of Commerce (BCC) has said.\n\nThe group\'s quarterly survey of companies found exports had picked up in the last three months of 2004 to their best levels in eight years. The rise came despite exchange rates being cited as a major concern. However, the BCC found the whole UK economy still faced "major risks" and warned that growth is set to slow. It recently forecast economic growth will slow from more than 3% in 2004 to a little below 2.5% in both 2005 and 2006.\n\nManufacturers\' domestic sales growth fell back slightly in the quarter, the survey of 5,196 firms found. Employment in manufacturing also fell and job expectations were at their lowest level for a year.\n\n"Despite some positive news for the export sector, there are worrying signs for manufacturing," the BCC said. "These results reinforce our concern over the sector\'s p

In [8]:
#pickle.dump(texts, open("sample_texts.pickle", "wb"))
texts = pickle.load(open("sample_texts.pickle", "rb"))

In [9]:
np.linalg.norm([1,2,3])
np.mean([[1,2,3],[1,2,3]], axis = 0)
vecs = np.array([[1,2,3], [1,3,4]])
def within_var(vecs):
    mean = np.mean(vecs, axis = 0)
    var = [np.sqrt(np.sum(np.square(v-mean))) for v in vecs]
    var = np.sum(var)/len(vecs)
    return var

x = ['rise', 'output', 'employment', 'sales', 'manufacturing', 'decelerate', 'levels', 'manufacturers', 'recovery', 'slowdown', 'investment', 'growth', 'sector', 'rates', 'costs', 'taxes', 'export', 'expectations', 'rate', 'margins', 'companies', 'increases', 'commerce', 'forecast', 'quarter', 'spending', 'economic', 'firms', 'increase', 'outlook', 'exports', 'quarterly', 'consumer', 'economy', 'sectors']
x_vecs = [word2vec[t] for t in x]
mean = np.mean(x_vecs, axis = 0)
dist = cosine_similarity(x_vecs, [mean])
dist = [i[0] for i in dist]
print(dist)
res = zip(dist, x)
sort_res = sorted(res,key=lambda res:res[0], reverse=True)
#for x in sort_res: print(x)
stopwords.words("english")+x
#for w, s in zip(dist, x):
    #print("{:.5f} {}".format(w, s))

[0.46623161, 0.5000754, 0.47485998, 0.61211121, 0.52359968, 0.48270348, 0.43089765, 0.47819805, 0.44630891, 0.66700476, 0.51780224, 0.71649152, 0.64512068, 0.5269596, 0.47951126, 0.42061377, 0.55034333, 0.4168452, 0.50337476, 0.49231279, 0.5324803, 0.54914927, 0.34307778, 0.53645945, 0.43091193, 0.46796137, 0.5972023, 0.49434227, 0.52344662, 0.54165626, 0.64347899, 0.54236472, 0.55759156, 0.65327764, 0.5773924]


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
true_text = BBCNews.data[-3]
#print(true_text)
#true_text = " ".join(stopwords.words("english")+x)
text = true_text
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
text = text.translate(translator).lower().split()
true_text = true_text.translate(translator).lower().split()
true_text = set([t for t in true_text if t in word2vec ])
start = time.time()
text = [t for t in text if t in word2vec ]
text = np.array(list(set(text)))
print(len(text))
num_clusters = math.ceil(np.sqrt(len(set(text))))
vecs = np.array([word2vec[w] for w in text])
for model in ["kmeans", "spectral", "ggm"]:
    similarity_matrix = cosine_similarity(vecs)+1
    for i in range(7, 8):
        if model == "kmeans":
            sc = KMeans(n_clusters = i, precompute_distances = True)
            sc.fit(vecs)
            distance_matrix = euclidean_distances(vecs)
        elif model == "spectral":
            sc = SpectralClustering(i, affinity='precomputed', assign_labels = "discretize", n_init=100)     
            sc.fit(similarity_matrix)
            distance_matrix = 2-similarity_matrix
        elif model == "ggm":
            dpgmm = mixture.BayesianGaussianMixture(n_components=i, covariance_type='full').fit(vecs)
            sc.labels_ = dpgmm.predict(vecs)

        print(len(set(sc.labels_)))
        silhoutte_scores = silhouette_samples(distance_matrix, sc.labels_, metric = 'precomputed')
        non_empty_clusters = np.array(list(set(sc.labels_)))
        indices = [np.where(sc.labels_ == j) if j in non_empty_clusters 
                                                           else [] for j in range(i)]
        sh_means = np.array([np.mean(silhoutte_scores[indices[j]]) for j in non_empty_clusters])
        for i in range(len(non_empty_clusters)):
            if True:
                sil_scores = silhoutte_scores[indices[i]]
                #print(within_var(vecs[indices[i]]))
                print(sh_means[i])
                words = text[indices[i]].tolist()
                output = ['{:.5f}'.format(s)+" "+w for s, w in zip(sil_scores, words)]
                #output = [w for w in words]
            print(output)
    print("*"*20)



144
7
-0.0159246
['-0.04449 versions', '0.05520 movies', '0.02457 clips', '-0.00389 screens', '0.00245 screen', '-0.09504 lets', '-0.07108 music', '0.02704 download', '-0.00768 files', '-0.02882 film', '-0.03344 video']
-0.0436062
['-0.04298 wireless', '-0.06002 work', '-0.06272 well', '-0.06312 than', '-0.00416 widely', '-0.03929 thought', '-0.03620 seen', '-0.03585 world', '-0.07191 its', '-0.08866 as', '-0.03391 dominance', '-0.05821 million', '-0.04045 sp', '-0.03477 will', '-0.04109 means', '-0.08513 also', '-0.01370 has', '-0.04110 be', '-0.04727 difference', '-0.06030 it', '-0.03652 no', '-0.03142 successful', '-0.03951 dual', '-0.03006 sensitive', '-0.04026 board', '-0.04332 preparing', '-0.00650 successor', '-0.03630 media', '-0.03964 cost', '-0.05890 more', '-0.04080 response', '-0.06900 have', '-0.04607 expected', '-0.04602 people', '-0.03932 an', '-0.04014 giant', '-0.02557 was', '-0.01765 adds', '-0.07521 so', '-0.04071 purpose', '-0.03740 said', '-0.01018 dominates', '-0.

# Topic soft cosine distance
build_vec() transforms a text into a vector.
Cluster_TSCD() computes the soft cosine similarity between two text vectors.

In [86]:
#similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
#for i in range(1, len(allTopics)):
#    for j in range(i):
#        x = time.time()
#        distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
#        similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
#        similarity_matrix[j, i] = similarity_matrix[i, j]
class PuncSplitTokenizer():
    """
        Map punctuations to space and then split, optional to remove predefined stopwords
    """
    def __init__(self, stopwords):
        self.translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
        self.stopwords = stopwords
    
    def __init__(self):
        self.translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
        self.stopwords = []
        
    def tokenize(self, text):
        tokens = text.translate(self.translator).lower().split()
        if self.stopwords:
            tokens = [t for t in tokens if t not in self.stopwords]
        return tokens

class TopicClusterGadget():
    """
        For vectorizing and computing document distance, based on word-topic-clusters in a document. 
        Implemented wmd_cosine, wmd_euclidean, and soft_cosine
    """
    def __init__(self, tokenizer, cls, distance_measure, word2vec):
        self.cls = cls
        self.tokenizer = tokenizer
        self.word2vec = word2vec
        self.distance_measure = distance_measure 
        if distance_measure == "wmd_euclidean":
            self.get_distance = DistanceTool.wmd_distance_euclidean(vec1, vec2)
        elif distance_measure == "wmd_cosine":
            self.get_distance = DistanceTool.wmd_distance_cosine(vec1, vec2)
        elif self.distance_measure == "soft_cosine":
            self.get_distance = DistanceTool.softcosine_distance(vec1, vec2)
    
    def tokenize(self, text):
        return self.tokenizer.tokenize(text)
    
    def vectorize_all(self, train_X, test_X):
        train_vecs = [self.encode(t) for t in train_X]
        test_vecs = [self.encode(t) for t in test_X]
        max_len = np.max(np.concatenate(([len(a) for a in train_vecs], [len(a) for a in test_vecs])))
        train_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in train_vecs])
        test_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in test_vecs])
        return train_vecs, test_vecs
        
    def encode(self, text):
        text = self.tokenize(text)
        text = [t for t in text if t in self.word2vec]
        word_counter = Counter(text)
        word_counts = np.array(list(word_counter.values()))
        unique_text = list(word_counter.keys())
        vecs = [self.word2vec[w] for w in unique_text]
        num_clusters = math.ceil(np.sqrt(len(unique_text)))
                                
        if self.cls == "kmeans": 
            model = KMeans(n_clusters = num_clusters, random_state=3425, precompute_distances = True)
            model.fit(vecs)
            distance_matrix = euclidean_distances(vecs)
            cluster_centers = model.cluster_centers_
        elif self.cls == "spectral":
            model = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1 
            model.fit(similarity_matrix) 
            distance_matrix = 2-similarity_matrix
        labels = model.labels_
        non_empty_clusters = np.array(list(set(labels)))                         
        indices = [np.where(labels == i) if i in non_empty_clusters 
                                                           else [] for i in range(num_clusters)]
        
        #silhoutte_scores = silhouette_samples(distance_matrix, labels, metric = 'precomputed')
        
        sizes = np.array([sum(word_counts[indices[i]]) for i in non_empty_clusters])
        weights = sizes/sum(sizes)                         
        return np.concatenate([[num_clusters], cluster_centers.flatten(), weights])
    
    def decode(self, a, b):
        num_clusters_a, num_clusters_b = int(a[0]), int(b[0])
        cut_a, cut_b = 1+300*num_clusters_a, 1+300*num_clusters_b
        topics1, topics2 = a[1:cut_a].reshape(-1,300), b[1:cut_b].reshape(-1,300)
        weights1, weights2 = a[cut_a:cut_a+num_clusters_a], b[cut_b:cut_b+num_clusters_b]
        allTopics = np.concatenate([topics1,topics2])
        d1 = np.concatenate([weights1,[0]*len(topics2)])
        d2 = np.concatenate([[0]*len(topics1),weights2])
        return d1, d2, allTopics

class DistanceTool():
    """
        Implemented wmd_cosine, wmd_euclidean, and soft_cosine
    """
    def softcosine_distance(self, vec1, vec2):
        d1, d2, allTopics = self.decode(vec1, vec2)
        similarity_matrix = cosine_similarity(allTopics)+1
        result = d1.T.dot(similarity_matrix).dot(d2)
        vec1 = d1.T.dot(similarity_matrix).dot(d1)
        vec2 = d2.T.dot(similarity_matrix).dot(d2)
        result /= (np.sqrt(vec1)*np.sqrt(vec2))
        return 1-result
    
    def wmd_distance_euclidean(self, vec1, vec2):
        return self.wmd_distance(vec1, vec2, euclidean_distances)
    
    def wmd_distance_cosine(self, vec1, vec2):
        return self.wmd_distance(vec1, vec2, cosine_distances)
    
    def wmd_distance(self, vec1, vec2, distance_func):
        d1, d2, allTopics = self.decode(vec1, vec2)
        distance_matrix = distance_func(allTopics, allTopics)
        return emd(d1, d2, distance_matrix)

class WordAverageGadget():
    """
        Averaging word vectors method
    """
    def __init__(self, tokenizer, word2vec):
        self.word2vec = word2vec
        self.tokenizer = tokenizer
        self.cls = "average"
        
    def tokenize(self, text):
        return self.tokenizer.tokenize(text)
    
    def vectorize_all(self, train_X, test_X):
        train_X = [self.encode(t) for t in train_X]
        test_X = [self.encode(t) for t in test_X]
        return train_X, test_X
    
    def encode(self, text):
        text = self.tokenize(text)
        return np.mean([self.word2vec[w] for w in text if w in self.word2vec], axis=0)
        
class WordMoversDistanceGadget():
    """
        The original word mover's distance
    """
    def __init__(self, tokenizer, word2vec):
        self.word2vec = word2vec
        self.tokenizer = tokenizer
        self.cls = "average"
        
    def tokenize(self, text):
        return self.tokenizer.tokenize(text)
    
    def vectorize_all(self, train_X, test_X):
        train_X = [self.encode(t) for t in train_X]
        test_X = [self.encode(t) for t in test_X]
        return train_X, test_X
    
    def encode(self, text):
        text = self.tokenize(text)
        return np.mean([self.word2vec[w] for w in text if w in self.word2vec], axis=0)
    
        
def euclidean(a, b):
    return np.sqrt(np.sum(np.square(a-b)))

def build_vec_auto(text, cls, strategy):
    """build vector represenetation of text
        cls: Clustering model
        strategy: used with spectral clustering 
            0: non-empy clusters, positive silhoutte scores, weighted average
            1: 
    """
    x = text
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
    text = text.translate(translator).lower().split()
    start = time.time()
    text = [t for t in text if t in word2vec and not t in stopwords.words("english")]
    if cls == "kmeans":
        num_clusters = math.ceil(np.sqrt(len(set(text))))
        vecs = [word2vec[w] for w in text]
        km = KMeans(n_clusters = num_clusters, random_state=3425, precompute_distances = True)
        km.fit(vecs)
        non_empty_clusters = np.array(list(set(km.labels_)))
        indices = [np.where(sc.labels_ == i) if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
        
        sizes = np.array([sum(word_counts[indices[i]]) for i in non_empty_clusters])
        sizes_weights = sizes/sum(sizes)
        cluster_sizes = np.array([len(np.where(km.labels_ == i)[0]) for i in range(num_clusters)])
        weights = cluster_sizes/sum(cluster_sizes)
        
        return np.concatenate([[num_clusters], km.cluster_centers_.flatten(), weights])
    elif cls == "spectral":
        if strategy == 0:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = np.array(list(word_counter.values()))
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = math.ceil(np.sqrt(len(unique_text)))
            vecs = np.array([word2vec[w] for w in unique_text])
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            silhoutte_scores = silhouette_samples(2-similarity_matrix, sc.labels_, metric = 'precomputed')
            non_empty_clusters = np.array(list(set(sc.labels_)))
            indices = [np.where(sc.labels_ == i) if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            sh_means = np.array([np.mean(silhoutte_scores[indices[i]]) for i in non_empty_clusters])
            pos_sh_means_indices = np.where(sh_means > 0)
            sh_means = sh_means[pos_sh_means_indices]
            good_clusters = non_empty_clusters[pos_sh_means_indices]
            sh_weights = np.array(sh_means/sum(sh_means))
            
            sizes = np.array([sum(word_counts[indices[i]]) for i in good_clusters])
            sizes_weights = sizes/sum(sizes)
            p = 0.3
            weights = p*sh_weights+(1-p)*sizes_weights
            #print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(weights)))
            centroids = np.array([np.mean(vecs[indices[i]], axis = 0) for i in good_clusters])
            rep = np.concatenate(([len(good_clusters)], centroids.flatten(), sizes_weights))
            return rep
        elif strategy == 1:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = list(word_counter.values())
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = math.ceil(np.sqrt(len(unique_text)))
            vecs = [word2vec[w] for w in unique_text]
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            silhoutte_scores = silhouette_samples(2-similarity_matrix, sc.labels_, metric = 'precomputed')
            silhoutte_contribution = np.multiply(word_counts, silhoutte_scores)
            total_contribution = sum(silhoutte_contribution)
            non_empty_clusters = list(set(sc.labels_))
            indices = [np.where(sc.labels_ == i)[0].tolist() if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            
            weights = np.array([sum(np.take(silhoutte_contribution, indices[i]))/total_contribution
                                                                        for i in non_empty_clusters])
            #print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(final_weights)))
            centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in non_empty_clusters])
            rep = np.concatenate([[len(non_empty_clusters)], centroids.flatten(), weights])
            #for i in range(len(weights)):
             #   print(weights[i])
              #  print(np.take(unique_text, indices[non_empty_clusters[i]]))
            #print("-----")
            return rep
        elif strategy == 2:
            word_counter = Counter(text)
            unique_text = list(word_counter.keys())
            word_counts = list(word_counter.values())
            if len(unique_text) < 3:
                unique_text = unique_text*2
                word_counts = word_counts*2
            num_clusters = 10#math.ceil(np.sqrt(len(unique_text)))
            vecs = [word2vec[w] for w in unique_text]
            sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                    assign_labels = "discretize", n_init=100)
            similarity_matrix = cosine_similarity(vecs)+1
            sc.fit(similarity_matrix)
            non_empty_clusters = list(set(sc.labels_))
            indices = [np.where(sc.labels_ == i)[0].tolist() if i in non_empty_clusters 
                                                               else [] for i in range(num_clusters)]
            sizes = np.array([sum(np.take(word_counts, indices[i])) for i in non_empty_clusters])
            centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in non_empty_clusters])
            rep = np.concatenate([[len(non_empty_clusters)], centroids.flatten(), [1/len(sizes)]*len(sizes)])
            return rep

def cluster_TSCD_auto(a, b):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    start = time.time()
    num_clusters_a, num_clusters_b = int(a[0]), int(b[0])
    cut_a, cut_b = 1+300*num_clusters_a, 1+300*num_clusters_b
    topics1, topics2 = a[1:cut_a].reshape(-1,300), b[1:cut_b].reshape(-1,300)
    weights1, weights2 = a[cut_a:cut_a+num_clusters_a], b[cut_b:cut_b+num_clusters_b]
    allTopics = np.concatenate([topics1,topics2])
    d1 = np.concatenate([weights1,[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),weights2])
    similarity_matrix = cosine_similarity(allTopics)
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    #print(time.time()-start)
    return np.clip(1-result, 0 , 1)

def cluster_TSCD(a, b, num_clusters):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    cut = 300*num_clusters
    topics1, topics2 = a[0:cut].reshape(-1,300), b[0:cut].reshape(-1,300)
    freq1, freq2 = a[cut:], b[cut:]
    allTopics = np.concatenate([topics1,topics2])
    
    d1 = np.concatenate([freq1/sum(freq1),[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),freq2/sum(freq2)])
    similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
    for i in range(1, len(allTopics)):
        for j in range(i):
            #distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
            similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    return np.clip(1-result, 0, 1)

def cluster_TSCD_raw(text1, text2, num_clusters):
    return cluster_TSCD(build_vec(text1, num_clusters), build_vec(text2, num_clusters), num_clusters)

def cluster_TSCD_raw_auto(text1, text2, cls, strategy):
    return cluster_TSCD_auto(build_vec_auto(text1, cls, strategy), build_vec_auto(text2, cls, strategy))

x = build_vec_auto(texts[-1], "spectral", 0)
x

NameError: name 'texts' is not defined

# Word average distance


In [23]:
def wad(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    #text = [t for t in text if not t in stopwords.words("english")]
    return np.mean([word2vec[w] for w in text if w in word2vec], axis=0)

def wordcount(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    text = [t for t in text if t in word2vec]
    return len(list(set(text)))

In [36]:
mydata = BBCNews_noised
train_X, test_X, train_y, test_y = train_test_split(mydata.data, mydata.label, test_size=0.3, random_state=42)

In [39]:

print("started KNN ")
start = time.time()
algo = "TSCD"
if algo == "wad":
    train_vecs = [ wad(t) for t in train_X]
    test_vecs = [ wad(t) for t in test_X]
elif algo == "LSA":
    LsaVectorizer = make_pipeline(TfidfVectorizer(max_features = 20000, use_idf = True, min_df=2)
                              ,TruncatedSVD(300))
    train_vecs = LsaVectorizer.fit_transform(train_X)
    test_vecs = LsaVectorizer.transform(test_X)
elif algo == "TSCD":
    train_vecs = [ build_vec_auto(t, 'spectral', 2) for t in train_X]
    test_vecs = [ build_vec_auto(t, 'spectral', 2) for t in test_X]
    max_len = np.max(np.concatenate(([len(a) for a in train_vecs], [len(a) for a in test_vecs])))
    train_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in train_vecs])
    test_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in test_vecs])
print(time.time() - start)
print("Vectorization finished")
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree'), cv = None, 
                    #param_grid={"n_neighbors": range(5,20,5)})
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree', 
 #                           metric=lambda a,b: cluster_TSCD_auto(a, b)), cv = None,
  #                         param_grid={"n_neighbors": range(5,25,5)})


started KNN 
287.6730680465698
Vectorization finished


In [40]:
if algo != "TSCD":
    nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree'), cv = None, 
                    param_grid={"n_neighbors": range(5,20,5)})
else:
    nbrs = KNeighborsClassifier(algorithm='ball_tree', n_neighbors = 10,
                                metric=lambda a,b: cluster_TSCD_auto(a, b))
nbrs.fit(train_vecs, train_y)
print(time.time() - start)
start = time.time()
#test_vecs = [build_vec(t, num_clusters) for t in news20_test.data]

print("started predicting")
start = time.time()
predicted = nbrs.predict(test_vecs)
print(accuracy_score(predicted, test_y))

354.6112232208252
started predicting
0.913173652695


# Benchmarking

In [87]:
datasets = [BBCNews]
models = [#TopicClusterDistanceGadget(PuncSplitTokenizer(), "kmeans", "soft_cosine", word2vec),
         WordAverageGadget(PuncSplitTokenizer(), word2vec)]

accuracy = []
for ds in datasets:
    for model in models:
        train_X, test_X, train_y, test_y = train_test_split(ds.data, ds.label, test_size=0.3, random_state=42)
        train_vecs, test_vecs = model.vectorize_all(train_X, test_X)
        if model.cls == "average":
            nbrs = KNeighborsClassifier(algorithm='ball_tree', n_neighbors = 10)
        else:
            nbrs = KNeighborsClassifier(algorithm='ball_tree', n_neighbors = 10,
                                metric=lambda a,b: model.get_distance(a, b))
        nbrs.fit(train_vecs, train_y)
        prediction = nbrs.predict(test_vecs)
        accuracy.append(accuracy_score(prediction, test_y))
        
print(accuracy)
        

[0.94161676646706582]
